In [1]:
!pip install datasets
!pip install accelerate -U
!pip install tensorflow
!pip install transformers
!pip install gdown
!pip install nvidia-cublas-cu12
!pip install torchvision
!pip install torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, AdamW, get_scheduler
import torch
import pandas as pd
from datasets import Dataset
import tensorflow as tf
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import gdown
import os

In [3]:
device = torch.device("cuda")

# **Dataset Loading**




In [4]:
df = pd.read_csv('C:\\Users\\Lenovo\\Desktop\\dataset.csv')
df.head()
print(df.shape)

(95, 2)


In [5]:
df['encoded_class'] = df['class'].astype('category').cat.codes

In [6]:
class_counts = df['encoded_class'].value_counts()
# Identify classes with only one sample
single_sample_classes = class_counts[class_counts == 1].index

# Filter out texts belonging to those classes
df = df[~df['encoded_class'].isin(single_sample_classes)]

In [7]:
df['encoded_class'] = df['class'].astype('category').cat.codes
data_texts = df['text'].to_list()
data_labels = df['encoded_class'].to_list()

In [8]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data_texts, data_labels, test_size=0.2, random_state=42, stratify=data_labels
)

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {'accuracy': accuracy}

def preprocess_data(texts, labels, tokenizer):
    encodings = tokenizer(texts, truncation=True, padding=True)
    dataset = Dataset.from_dict({**encodings, 'labels': labels})
    return dataset

# **Model Training and Saving**

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['encoded_class'].unique()))
model.to(device)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold = 0
best_loss = float('inf')
accuracy_threshold = 0.8
best_model_dir = None
for train_index, val_index in kf.split(train_texts):
    fold += 1
    print(f"Training fold {fold}...")
    print((train_index, val_index))

    train_texts_fold = [train_texts[i] for i in train_index]
    train_labels_fold = [train_labels[i] for i in train_index]
    val_texts_fold = [train_texts[i] for i in val_index]
    val_labels_fold = [train_labels[i] for i in val_index]

    train_dataset = preprocess_data(train_texts_fold, train_labels_fold, tokenizer)
    val_dataset = preprocess_data(val_texts_fold, val_labels_fold, tokenizer)

    training_args = TrainingArguments(
        output_dir=f'./results/fold_{fold}',
        num_train_epochs=30,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        warmup_steps=10,
        weight_decay=0.01,
        logging_dir=f'./logs/fold_{fold}',
        evaluation_strategy="steps",
        eval_steps=5,
        save_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        learning_rate=2e-5,
    )

    optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

    num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
    scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=training_args.warmup_steps,
        num_training_steps=num_training_steps
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        optimizers=(optimizer, scheduler),
        compute_metrics=compute_metrics
    )

    trainer.train()

    eval_metrics = trainer.evaluate()
    print(f"Fold {fold} evaluation metrics: {eval_metrics}")

    trainer.save_model(f'./results/best_model_fold_{fold}')
    tokenizer.save_pretrained(f'./results/best_model_fold_{fold}')

    if eval_metrics["eval_accuracy"] >= accuracy_threshold:
        if eval_metrics["eval_loss"] < best_loss:
            best_loss = eval_metrics["eval_loss"]
            best_model_dir = f'./results/best_model_fold_{fold}'

if best_model_dir:
    model = DistilBertForSequenceClassification.from_pretrained(best_model_dir)
    model.save_pretrained('./results/best_model')
    tokenizer.save_pretrained('./results/best_model')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 1...
(array([ 1,  2,  3,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71]), array([ 0,  4, 10, 18, 28, 34, 63, 72]))


C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
5,No log,2.461210,0.000000
10,No log,2.462306,0.125000
15,No log,2.450592,0.125000
20,No log,2.431507,0.125000
25,No log,2.420792,0.125000
30,No log,2.411692,0.125000
35,No log,2.416142,0.125000
40,No log,2.388106,0.125000
45,No log,2.343949,0.125000
50,No log,2.323193,0.125000


Fold 1 evaluation metrics: {'eval_loss': 0.5770097970962524, 'eval_accuracy': 0.875, 'eval_runtime': 0.6774, 'eval_samples_per_second': 11.81, 'eval_steps_per_second': 5.905, 'epoch': 30.0}
Training fold 2...
(array([ 0,  1,  2,  3,  4,  6,  7,  8, 10, 11, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72]), array([ 5,  9, 12, 22, 31, 45, 55, 65]))


...

Step,Training Loss,Validation Loss,Accuracy
5,No log,0.000002,1.000000
10,No log,0.000002,1.000000
15,No log,0.000002,1.000000
20,No log,0.000002,1.000000
25,No log,0.000002,1.000000
30,No log,0.000002,1.000000
35,No log,0.000002,1.000000
40,No log,0.000002,1.000000
45,No log,0.000002,1.000000
50,No log,0.000002,1.000000


Fold 9 evaluation metrics: {'eval_loss': 1.3794199276162544e-06, 'eval_accuracy': 1.0, 'eval_runtime': 0.5939, 'eval_samples_per_second': 11.786, 'eval_steps_per_second': 6.735, 'epoch': 30.0}
Training fold 10...
(array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
       19, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56,
       57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]), array([ 2, 14, 20, 21, 23, 51, 60]))


C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
5,No log,0.000001,1.000000
10,No log,0.000001,1.000000
15,No log,0.000001,1.000000
20,No log,0.000001,1.000000
25,No log,0.000001,1.000000
30,No log,0.000001,1.000000
35,No log,0.000001,1.000000
40,No log,0.000001,1.000000
45,No log,0.000001,1.000000
50,No log,0.000001,1.000000


Fold 10 evaluation metrics: {'eval_loss': 9.196136261380161e-07, 'eval_accuracy': 1.0, 'eval_runtime': 0.4527, 'eval_samples_per_second': 15.463, 'eval_steps_per_second': 8.836, 'epoch': 30.0}


# **Model Loading**

In [15]:
best_model_dir = './best_model'
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(best_model_dir)
model_fine_tuned = DistilBertForSequenceClassification.from_pretrained(best_model_dir)
#model_fine_tuned.to(device)

In [ ]:
test = df['text'][20]     #index: 0 -> 92
print(test)

predict_input = tokenizer_fine_tuned.encode(
    test,
    truncation = True,
    padding = True,
    return_tensors = 'pt'
)

with torch.no_grad():
    output = model_fine_tuned(predict_input)[0]

prediction_value = torch.argmax(output, dim=1).numpy()[0]
df['class'] = df['class'].astype('category')

classes = df['class'].cat.categories

print(classes[prediction_value])

治験にご参加いただき ありがとうございます 治験への参加は、社会全体の健康増進と医学の進歩につながる 大変素晴らしい献身的な行動であると、私たちは確信しております。 このカードはアストラゼネカ社の治験番号：D081CC00006にご参加いただいた方のためものです。このカードは、一般の方々に 治験参加に関する情報を提供するために設立された独立非営利組織である「Center for Information & Study on Clinical Research Participation（CISCRP）」により作成されました。CISCRPは、治験の参加者募集や治験の実施には関与しておりません。 治験への参加 この治験にご参加いただいたあなたや、ご参加いた だいている多くの方々に、心より感謝申し上げます。 あなたには、アストラゼネカが依頼し、Breast International Group、Frontier Science & Technology Research Foundation、およびNRG Oncologyとの提携で調整された治験に参加いただきました。治験は、世界中の医師、科学者、 治験コーディネーター、参加者の皆様等から成る大規模なチームによって実施されます。あなたの参加のおかげで、 この薬オラパリブが、すでに手術や化学療法を受けた成人の乳がん患者さんに役立つかどうかを調べることができます。 治験が完了すると、結果の概要をwww.trialsummaries.comで閲覧いただけるようになります。詳細については、こちらのウェ ブサイトでご確認ください。登録手続きをしていただくと、治験結果の概要が準備でき次第、電子メールで通知いたします。 インターネットを使えない場合、または結果を印刷したものが必要な場合は、治験スタッフにお知らせください。治験に参加 されるすべての方々が、医学の進歩のために重要な役割を担っていることを誇りに思っていただけるよう願っています。 入手可能になった際、治験結果の要約を受け取るには、 www.trialsummaries.com にアクセスします Dコード D081CC00006を検索 電子メール通知を ご登録ください JA-JP 第1.0版 2023年8月28日 www.astrazeneca.com 
C14_01


# **Model Testing**

In [16]:
def calculate_accuracy(preds, labels):
    return accuracy_score(labels, preds)

In [17]:
test_dataset = preprocess_data(test_texts, test_labels, tokenizer_fine_tuned)
data_collator = DataCollatorWithPadding(tokenizer_fine_tuned)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)
model_fine_tuned.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v for k, v in batch.items() if k != 'labels'}
        labels = batch['labels']
        outputs = model_fine_tuned(**inputs)
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

accuracy = calculate_accuracy(all_preds, all_labels)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8421052631578947
